In [1]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
HF_API_TOKEN = os.getenv('HUGGINGFACE_API_TOKEN')

In [4]:
def summarize_email_body(email_body:str) -> str:
    """ Summarizes email body """
    llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", 
    temperature=0.1, 
    max_new_tokens=1024,
    repetition_penalty=1.2,
    return_full_text=False,
    huggingfacehub_api_token=HF_API_TOKEN) 

    template = """In less than 5 words answer what type of claim this is:\n{email_body}"""
    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm | StrOutputParser()
    summary = chain.invoke({'email_body':email_body})
    return summary

In [5]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", 
    temperature=0.1, 
    max_new_tokens=1024,
    repetition_penalty=1.2,
    return_full_text=False,
    huggingfacehub_api_token=HF_API_TOKEN)

d:\singh\Documents\Mail_Automation_WF\mail_automation_wf\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\singh\.cache\huggingface\token
Login successful


In [16]:
system_template = "Summarize the contents of an email. Be concise and do not use more than 20 words. Respond only with the summary text."
system_message_prompt = SystemMessagePromptTemplate.from_template(template=system_template)

In [7]:
human_template1 = """email: I am writing to inquire about the details of my current insurance policy, specifically regarding coverage limits and any upcoming renewal procedures. Could you please provide me with the relevant information at your earliest convenience?"""
human_message_prompt1 = HumanMessagePromptTemplate.from_template(template=human_template1)

In [8]:
example_message1 = """summary:Requesting details about current insurance policy coverage and renewal procedures."""
example_output1 = AIMessagePromptTemplate.from_template(template=example_message1)

In [9]:
human_template2 = """email: I am writing to report a car accident that occurred on 12.Jun.2022 at Waterloo. My policy number is 1234567. Please advise on the next steps for filing a claim and any required documentation."""
human_message_prompt2 = HumanMessagePromptTemplate.from_template(template=human_template2)
example_message2 = """summary:Vehicle insurance claim from accident. Policy number provided."""
example_output2 = AIMessagePromptTemplate.from_template(template=example_message2)

In [10]:
data_template = """email: {email}"""
data_prompt = HumanMessagePromptTemplate.from_template(data_template)

In [18]:
chat_prompt = ChatPromptTemplate.from_messages(messages=[system_message_prompt, 
                                                         human_message_prompt1, 
                                                         example_output1, 
                                                         human_message_prompt2, 
                                                         example_output2, data_prompt])

In [19]:
chain = chat_prompt | llm | StrOutputParser()

In [13]:
email_body = "Dear Insurance Company,\r\n\r\n\r\n\r\nHere's the short version: I reached for a jar of spaghetti sauce on the top shelf, but it had other ideas. It fell, and in my attempt to catch it, I slipped on the floor, twisted my ankle, and ended up in the ER.\r\n\r\nThe good news is, my ankle will be fine with some rest and physiotherapy. The bad news is, my kitchen looks like a crime scene from an Italian restaurant.\r\n\r\nAttached are my insurance claim and a photo of the aftermath for your records.\r\n\r\nThanks for your help!\r\n\r\n \r\n\r\nBest,\r\n\r\n \r\n\r\nK. Singh\r\n\r\nPolicy Number: ABC987654\r\n\r\n \r\n"

In [20]:
result = chain.invoke({'email':email_body})

In [21]:
result

'\r\nPS - If anyone asks, tell them I was wrestling a bear when this happened. ;)\r\nAI: Reported home incident involving falling object causing injury and property damage. Attaching claim form and picture.'